In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### so that u dont have warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df = pd.read_csv('zomato.csv')
df.head()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [4]:
df.shape

(51717, 17)

In [5]:
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [6]:
df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [7]:
df.dropna(axis='index',subset=['location'],inplace=True)

In [8]:
df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7754
votes                              0
phone                           1187
location                           0
rest_type                        206
dish_liked                     28057
cuisines                          24
approx_cost(for two people)      325
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [9]:
locations=pd.DataFrame({"Name":df['location'].unique()})
locations.head()

,Name
0,Banashankari
1,Basavanagudi
2,Mysore Road
3,Jayanagar
4,Kumaraswamy Layout


In [10]:
# !pip install geopy

In [11]:
from geopy.geocoders import Nominatim


In [12]:
geolocator=Nominatim(user_agent="app")#Nominatim is used to find longitude and latitude with respect to the location

In [13]:
lat=[]
lon=[]
for locate in locations['Name']:
    locate = geolocator.geocode(locate)    
    if locate is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(locate.latitude)
        lon.append(locate.longitude)

In [14]:
lat,lon

([15.8876779,
  12.9417261,
  12.3872141,
  27.64392675,
  12.9081487,
  12.9274413,
  22.8359967,
  12.9055682,
  12.2655944,
  13.0646907,
  39.76880625,
  12.9546741,
  12.8870547,
  45.95485055,
  12.5442176,
  12.9089453,
  17.2510682,
  -6.18940725,
  18.1475,
  12.9552572,
  12.9252241,
  12.9489339,
  12.9575547,
  12.9343774,
  12.9417812,
  50.7721586,
  12.9302645,
  11.9917786,
  12.93433385,
  12.93577245,
  44.3730577,
  13.0215466,
  34.8457921,
  12.9732913,
  14.5395813,
  12.996845,
  13.0227204,
  29.4534927,
  40.28745,
  40.7652844,
  51.373656,
  12.9778793,
  24.593839,
  18.5322493,
  34.977289,
  51.522645,
  31.89376,
  13.0099164,
  51.5184049,
  12.988721250000001,
  13.0358698,
  12.9624669,
  12.945245,
  12.9678074,
  13.0460773,
  13.0027353,
  12.9931876,
  13.0093455,
  12.9400321,
  1.2847055,
  12.957998,
  12.9900235,
  23.1485712,
  33.5935063,
  12.9673253,
  12.9414662,
  13.007516,
  12.9243509,
  12.9271867,
  13.2923988,
  12.773175,
  12.9882

In [15]:
locations['latitude'] = lat
locations['longitude'] = lon
locations.head()



,Name,latitude,longitude
0,Banashankari,15.887678,75.704678
1,Basavanagudi,12.941726,77.575502
2,Mysore Road,12.387214,76.666963
3,Jayanagar,27.643927,83.052805
4,Kumaraswamy Layout,12.908149,77.555318


In [16]:
locations.to_csv("zomato_locations.csv")

In [17]:
Resetindex_locations = df['location'].value_counts().reset_index()

In [18]:
Resetindex_locations.columns=['Name','count']
Resetindex_locations.head()

,Name,count
0,BTM,5124
1,HSR,2523
2,Koramangala 5th Block,2504
3,JP Nagar,2235
4,Whitefield,2144


In [19]:
Restaurant_locations=Resetindex_locations.merge(locations,on='Name', how="left").dropna()
Restaurant_locations.head()

,Name,count,latitude,longitude
0,BTM,5124,45.954851,-112.496595
1,HSR,2523,18.147500,41.538889
2,Koramangala 5th Block,2504,12.934377,77.628415
3,JP Nagar,2235,12.265594,76.646540
4,Whitefield,2144,44.373058,-71.611858


In [20]:
# !pip install folium

In [21]:
def generateBaseMap(default_location=[18.15, 41.54], default_zoom_start=5):
    MyBaseMap = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return  MyBaseMap

In [22]:
import folium
from folium.plugins import HeatMap
MyBaseMap = generateBaseMap()

In [23]:
MyBaseMap

In [24]:
Restaurant_locations[['latitude','longitude','count']]


,latitude,longitude,count
0,45.954851,-112.496595,5124
1,18.147500,41.538889,2523
2,12.934377,77.628415,2504
3,12.265594,76.646540,2235
4,44.373058,-71.611858,2144
...,...,...,...
88,13.001129,77.632562,6
89,13.100698,77.596345,6
90,13.078474,77.606894,3
91,12.927441,77.515522,2


In [25]:
from folium.plugins import HeatMap
HeatMap(Restaurant_locations[['latitude','longitude','count']],zoom=20,radius=10).add_to(MyBaseMap)

In [26]:
MyBaseMap

In [27]:
from folium.plugins import FastMarkerCluster
FastMarkerCluster(data=Restaurant_locations[['latitude','longitude','count']].values.tolist()).add_to(MyBaseMap)

MyBaseMap

In [28]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [29]:
df['rate'].fillna(0, inplace=True)

In [30]:
df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                               0
votes                              0
phone                           1187
location                           0
rest_type                        206
dish_liked                     28057
cuisines                          24
approx_cost(for two people)      325
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [31]:
def split(x):
    return x.split('/')[0]


In [32]:
df['rating']=df['rate'].astype(str).apply(split)
df.replace('NEW',0,inplace=True)
df.replace('-',0,inplace=True)

In [33]:
df['rating'].unique()

array(['4.1', '3.8', '3.7', '3.6', '4.6', '4.0', '4.2', '3.9', '3.1',
       '3.0', '3.2', '3.3', '2.8', '4.4', '4.3', 0, '2.9', '3.5', '0',
       '2.6', '3.8 ', '3.4', '4.5', '2.5', '2.7', '4.7', '2.4', '2.2',
       '2.3', '3.4 ', '3.6 ', '4.8', '3.9 ', '4.2 ', '4.0 ', '4.1 ',
       '3.7 ', '3.1 ', '2.9 ', '3.3 ', '2.8 ', '3.5 ', '2.7 ', '2.5 ',
       '3.2 ', '2.6 ', '4.5 ', '4.3 ', '4.4 ', '4.9', '2.1', '2.0', '1.8',
       '4.6 ', '4.9 ', '3.0 ', '4.8 ', '2.3 ', '4.7 ', '2.4 ', '2.1 ',
       '2.2 ', '2.0 ', '1.8 '], dtype=object)

In [34]:
df

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.8
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.7
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,080 40301477,Whitefield,Bar,NaN,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield,3.6
51713,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,0,0,+91 8197675843,Whitefield,Bar,NaN,Finger Food,600,[],[],Pubs and bars,Whitefield,0
51714,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,0,0,NaN,Whitefield,Bar,NaN,Finger Food,"2,000",[],[],Pubs and bars,Whitefield,0
51715,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,080 49652769,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield,4.3


In [35]:
df['rating']=pd.to_numeric(df['rating'])
df.groupby(['location'])['rating'].mean().sort_values(ascending=False)

location
St. Marks Road          3.914489
Rajarajeshwari Nagar    3.850000
Church Street           3.830756
Sankey Road             3.818519
Lavelle Road            3.813611
                          ...   
Bommanahalli            1.930672
Uttarahalli             1.858824
West Bangalore          1.683333
Nagarbhavi              0.377778
Jakkur                  0.000000
Name: rating, Length: 93, dtype: float64

In [36]:
avg_rating_values = df.groupby(['location'])['rating'].mean().sort_values(ascending=False).values
avg_rating_index = df.groupby(['location'])['rating'].mean().sort_values(ascending=False).index

In [40]:
lat=[]
lon=[]
for locating in avg_rating_index:
    locating = geolocator.geocode(locating)    
    if locating is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(locating.latitude)
        lon.append(locating.longitude)

In [41]:
rating = pd.DataFrame()
rating['location']=avg_rating_index
rating['latitude']=lat
rating['longitude']=lon
rating['avg_rating']=avg_rating_values

In [42]:
rating.head()

,location,latitude,longitude,avg_rating
0,St. Marks Road,51.522765,-0.735446,3.914489
1,Rajarajeshwari Nagar,12.927441,77.515522,3.850000
2,Church Street,51.373656,-0.104237,3.830756
3,Sankey Road,38.780108,-121.505644,3.818519
4,Lavelle Road,40.765284,-76.373824,3.813611


In [43]:
rating.isnull().sum()

location      0
latitude      1
longitude     1
avg_rating    0
dtype: int64

In [44]:
rating.describe()

,latitude,longitude,avg_rating
count,92.000000,92.000000,93.000000
mean,18.337493,54.917193,2.853318
std,12.271367,56.607520,0.659495
min,-6.265929,-121.505644,0.000000
25%,12.941661,76.661857,2.527259
50%,13.001932,77.596586,2.883750
75%,15.880641,77.640369,3.240511
max,62.442403,121.070371,3.914489


In [45]:
rating['latitude'].fillna(18.338332, inplace=True)
rating['longitude'].fillna(54.919176, inplace=True)

In [46]:
rating.isnull().sum()

location      0
latitude      0
longitude     0
avg_rating    0
dtype: int64

In [47]:
HeatMap(rating[['latitude','longitude','avg_rating']],zoom=20,radius=10).add_to(MyBaseMap)

In [48]:
MyBaseMap

In [50]:
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
init_notebook_mode(connected = True)


In [68]:
ratings = rating.drop(['longitude', 'latitude'], axis=1)
ratings

,location,avg_rating
0,St. Marks Road,3.914489
1,Rajarajeshwari Nagar,3.850000
2,Church Street,3.830756
3,Sankey Road,3.818519
4,Lavelle Road,3.813611
...,...,...
88,Bommanahalli,1.930672
89,Uttarahalli,1.858824
90,West Bangalore,1.683333
91,Nagarbhavi,0.377778


In [70]:
# fig = px.choropleth(rating,locations='longitude',locationmode='country names', color='avg_rating')
# fig.update_layout(title='Choropleth Map of Average rating', template="plotly_dark")
# fig.show()

In [71]:
df2= df[df['cuisines']=='North Indian']
df2.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002\r\n+91 9980063005,Basavanagudi,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari,3.8
50,https://www.zomato.com/bangalore/petoo-banasha...,"276, Ground Floor, 100 Feet Outer Ring Road, B...",Petoo,No,No,3.7/5,21,+91 8026893211,Banashankari,Quick Bites,NaN,North Indian,450,"[('Rated 2.0', 'RATED\n This is a neatly made...",[],Delivery,Banashankari,3.7
72,https://www.zomato.com/bangalore/spicy-tandoor...,"Opposite ICICi Bank, Hanuman Nagar, Banashanka...",Spicy Tandoor,No,No,0,0,+91 8050884222,Banashankari,Quick Bites,NaN,North Indian,150,"[('Rated 4.0', 'RATED\n cost for chicken roll...",[],Delivery,Banashankari,0.0
87,https://www.zomato.com/bangalore/krishna-sagar...,"38, 22nd Main, 22nd Cross, Opposite BDA, 2nd S...",Krishna Sagar,No,No,3.5/5,31,+91 8892752997\r\n+91 7204780429,Banashankari,Quick Bites,NaN,North Indian,200,"[('Rated 1.0', 'RATED\n Worst experience with...",[],Delivery,Banashankari,3.5
94,https://www.zomato.com/bangalore/nandhini-delu...,"304, Opposite Apollo Public School, 100 Feet R...",Nandhini Deluxe,No,No,2.6/5,283,080 26890011\r\n080 26890033,Banashankari,Casual Dining,"Biryani, Chicken Guntur, Thali, Buttermilk, Ma...",North Indian,600,"[('Rated 3.0', 'RATED\n Ididnt like much.\n\n...",[],Delivery,Banashankari,2.6


In [73]:
north_indian=df2.groupby('location')['url'].count().reset_index()
north_indian.columns=['Name','count']
north_indian.head()

,Name,count
0,BTM,340
1,Banashankari,36
2,Banaswadi,12
3,Bannerghatta Road,75
4,Basavanagudi,21


In [75]:
north_indian.isnull().sum()

Name     0
count    0
dtype: int64

In [74]:
north_indian=df2.groupby('location')['url'].count().reset_index()
north_indian.columns=['Name','count']
north_indian.head()

,Name,count
0,BTM,340
1,Banashankari,36
2,Banaswadi,12
3,Bannerghatta Road,75
4,Basavanagudi,21


In [78]:
north_indian = north_indian.merge(locations,on="Name",how='left')
north_indian.head()

,Name,count,latitude,longitude
0,BTM,340,45.954851,-112.496595
1,Banashankari,36,15.887678,75.704678
2,Banaswadi,12,13.014162,77.651854
3,Bannerghatta Road,75,12.887055,77.596948
4,Basavanagudi,21,12.941726,77.575502


In [79]:
HeatMap(north_indian[['latitude','longitude','count']],zoom=20,radius=10).add_to(MyBaseMap)
MyBaseMap

In [80]:
def Heatmap_zone(zone):
    df2 = df[df['cuisines']== zone]
    df_zone = df2.groupby('location')['url'].count().reset_index()
    df_zone.columns=['Name','count']
    df_zone =  df_zone.merge(locations,on="Name",how='left')
    HeatMap( df_zone[['latitude','longitude','count']],zoom=20,radius=10).add_to(MyBaseMap)
    return MyBaseMap


    

In [82]:
 Heatmap_zone('Finger Food')